#关于Agent
需求：
1、可以切换不同的用户，各自的聊天记录和对话环境（room）互不干扰
2、可以加载不同的prompt扮演不同的角色。
3、保持上下文，最好能记住关键词，能提炼关键词记住关键词。或者，有无穷大的memery
4、给tools留下空间。要有两个tools
   4.1 能看网页，在上下文自动提取网页地址并浏览获取信息。
   4.2 能搜索信息，比如搜索百度。
   4.3 能上传文件，并基于文件对话。比如上传佛经扮演上师，上传客户指导扮演客服、扮演老师讲题。等等
   4.4 能画图，自己生成咒语，出图。api
   
解决方案：
1、必须是Agent，要有tools和memery的预留空间。
2、prompte template可变。
3、memery基于key可以替换，可以存储，可以提取。
4、支持语义搜索数据库，存储上传的文件和对话记录。
5、tools。
6、需要自主功能吗？比如总结整个文件的内容，等等。一个文件片段包含不下的内容，就需要自主功能来解决。

#矢量数据库 https://www.activeloop.ai/ DeepLake


In [ ]:
from langchain.memory import ChatMessageHistory
from langchain.schema import messages_from_dict, messages_to_dict

history = ChatMessageHistory()

history.add_user_message("嗨!")
history.add_ai_message("咋了?")

history.add_user_message("你瞅啥!")
history.add_ai_message("瞅你咋地?")



dicts = messages_to_dict(history.messages)

dicts 


In [ ]:

history.messages = messages_from_dict(dicts)
history.messages

In [ ]:
fakeDB = [
    {'userid':1,'history':[{'type': 'human', 'data': {'content': '嗨!', 'additional_kwargs': {}}},
                            {'type': 'ai', 'data': {'content': '咋了?', 'additional_kwargs': {}}},
                            {'type': 'human', 'data': {'content': '你瞅啥!', 'additional_kwargs': {}}},
                            {'type': 'ai', 'data': {'content': '瞅你咋地?', 'additional_kwargs': {}}}]
                            },
    {'userid':2,'history':[{'type': 'ai', 'data': {'content': '来了老弟!', 'additional_kwargs': {}}},
                            {'type': 'human', 'data': {'content': '来了嫂子', 'additional_kwargs': {}}},
                            {'type': 'ai', 'data': {'content': '今天吃点儿啥呀!', 'additional_kwargs': {}}},
                            {'type': 'human', 'data': {'content': '能有啥呀，整个煎饼果子吧', 'additional_kwargs': {}}}]
                            },


]

def get_history_by_uid(usrid):
    for db in fakeDB:
        if db['userid']  == usrid:
            return db['history']
        

db = get_history_by_uid(2)
db

In [ ]:
history.clear()
history.messages = messages_from_dict(get_history_by_uid(2))

history.messages

In [11]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationBufferWindowMemory


import json
from langchain.schema import messages_from_dict, messages_to_dict

memory=ConversationBufferWindowMemory(k=6)

memory.chat_memory.add_user_message("嘿!")
memory.chat_memory.add_ai_message("咋了?")
memory.chat_memory.dict()
#memory.save_context()

{'messages': [{'content': '嘿!', 'additional_kwargs': {}},
  {'content': '咋了?', 'additional_kwargs': {}}]}

In [33]:
memory.load_memory_variables({})


{'history': 'Human: 嘿!\nAI: 咋了?'}

In [51]:
memory.clear()
#模拟切换用户，换memory_key作为用户索引，chat_memory作为用户的对话上下文。
memory.memory_key = "user_2" 
memory.chat_memory.messages =  messages_from_dict(get_history_by_uid(2))
memory.dict()

{'chat_memory': {'messages': [{'content': '来了老弟!', 'additional_kwargs': {}},
   {'content': '来了嫂子?', 'additional_kwargs': {}},
   {'content': '今天吃点儿啥呀!', 'additional_kwargs': {}},
   {'content': '能有啥呀，整个煎饼果子吧', 'additional_kwargs': {}}]},
 'output_key': None,
 'input_key': None,
 'return_messages': False,
 'human_prefix': 'Human',
 'ai_prefix': 'AI',
 'memory_key': 'user_2',
 'k': 6}

In [ ]:
memory.load_memory_variables({})

In [5]:
from langchain.utilities import TextRequestsWrapper
from langchain.agents import Tool

requests = TextRequestsWrapper()

request_tool = Tool(
    name = "requests",
    func =  requests.get,
    description="用于获取网页内容，送入参数是网页的url地址，返回网页内容",

)

In [4]:
requests.get("https://www.hrfjw.com/fojing/liuzutanjing/198840_2.html")

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\r\n<html xmlns="http://www.w3.org/1999/xhtml">\r\n<head>\r\n<meta http-equiv="Content-Type" content="text/html; charset=gb2312" />\r\n<meta http-equiv="Cache-Control" content="no-transform" />\r\n<meta http-equiv="Cache-Control" content="no-siteapp" />\r\n<meta name="applicable-device" content="pc">\r\n<meta name="mobile-agent"content="format=xhtml;url=https://m.hrfjw.com/fojing/liuzutanjing/198840.html" />\r\n<meta name-equiv="mobile-agent" content="format=html5;url=https://m.hrfjw.com/fojing/liuzutanjing/198840.html" />\r\n<title>¡¶Áù×æÌ³¾\xad¡·¶þÊ®ËÄ¾«ÂÛ - »ªÈË·ð½ÌÍø</title>\r\n<meta name="keywords" content="Áù×æÌ³¾\xad" />\r\n<meta name="description" content="¡¡¡¡·ð½Ììø×Ú´Ó¶þÊ®°Ë×æÆÐÌá´ïÄ¦µ½ìø×ÚÈýÊ®ËÄ×æ£¬¾\xad¶«ÍÁÖÐ»ªÁù×æ»ÛÄÜ´óÊ¦µÄ·¨Âö´«³ÐÓë×¿Ô½ºëÑï£¬¿ª´´ÁËìø×Ú¡°Ò»»¨ÎåÒ¶¡±µÄÐËÊ¢¾ÖÃæ¡£ÔÚ´óÌÆÊ¢ÊÀÊ±ÆÚ£¬·ð½Ì³É¹¦µØÊµÏÖÁË×ÔÎÒ¸ïÐÂ£¬¾\xadÀú´úìø×Ú×æÊ¦´óµÂµÄ²»Ð¸Å¬Á¦£¬½ø¶

In [ ]:
from langchain.agents import Tool
from langchain.agents import AgentType
from langchain.memory import ConversationBufferMemory
from langchain import OpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.agents import initialize_agent

In [ ]:
search = SerpAPIWrapper()
tools = [
    Tool(
        name = "Current Search",
        func=search.run,
        description="useful for when you need to answer questions about current events or the current state of the world"
    ),
]

In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history")

In [ ]:
llm=OpenAI(temperature=0)
agent_chain = initialize_agent(tools, llm, agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION, verbose=True, memory=memory)

agent_chain.run(input="hi, i am bob")

# 会使用工具的Agent

LLM Agent 使用语言模型的Agent .

In [ ]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain import OpenAI, SerpAPIWrapper, LLMChain
from langchain.chat_models import ChatOpenAI
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish
import re

In [ ]:
# Set up the base template
template = """尽最大努力回答Question, 用哥们儿一样的语气说话.你可以使用以下工具:

{tools}

使用以下公式:

Question: 输入的你必须回答的问题
Thought: 你应该慎重考虑你要做什么
Action: 采取的行动，行动必须是后面几个工具之一 [{tool_names}]
Action Input: 行动的输入
Observation: 行动的结果
... (这四个过程 Thought/Action/Action Input/Observation 可以重复 N 次)
Thought:我知道最终答案了。
Final Answer: 最初问题的最终答案

开始! 记住，给出最终答案的时候，要像兄弟哥们儿那样的语气说话，多用"哈哈"

Question: {input}
{agent_scratchpad}"""

In [ ]:
# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]
    
    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)

In [ ]:
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"]
)

In [ ]:
class CustomOutputParser(AgentOutputParser):
    
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

In [ ]:
output_parser = CustomOutputParser()

In [ ]:
# Set up LLM
# Choose the LLM you want to use!

llm = ChatOpenAI(temperature=0)

In [ ]:
#Set up the Agent
#We can now combine everything to set up our agent
# LLM chain consisting of the LLM and a prompt
llm_chain = LLMChain(llm=llm, prompt=prompt)

tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    stop=["\nObservation:"], 
    allowed_tools=tool_names
)

# Use the Agent
Now we can use it!

In [ ]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.run("How many people live in canada as of 2023?")

# 下面讨论如何增加memery

想增加memery,需要增加两个步骤
* 在 custom prompt 增加 chat_history
* 给 agent executor 增加一个 memory 对象

In [ ]:
# 带history的模板
template_with_history = """尽你最大所能回答question, 但是说话语气要像兄弟哥们儿一样. 
你可以使用的工具包括:

{tools}

使用如下公式:

Question: 你必须回答的问题
Thought: 你应该谨慎思考你有做什么
Action: 采取的行动, 必须是后面列出的行动之一 [{tool_names}]
Action Input: 行动的输入
Observation: 行动的结果
... (这四个步骤的循环 Thought/Action/Action Input/Observation 可以重复 N 次)
Thought: 我找到最终答案了
Final Answer: 最初问题的最终答案

开始! 记住，在给出最终答案的时候，要向哥们儿兄弟那样说话.多用"哈哈"

以前对话的历史记录:
{history}

New question: {input}
{agent_scratchpad}"""

In [ ]:
prompt_with_history = CustomPromptTemplate(
    template=template_with_history,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps", "history"]
)

In [ ]:
llm_chain = LLMChain(llm=llm, prompt=prompt_with_history)

In [ ]:
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    stop=["\nObservation:"], 
    allowed_tools=tool_names
)

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

memory=ConversationBufferWindowMemory(k=2)

In [ ]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, memory=memory)

In [ ]:
agent_executor.run("How many people live in canada as of 2023?")

In [ ]:
agent_executor.run("how about in mexico?")

Custom MRKL Agent

https://zhuanlan.zhihu.com/p/526713337 
符号主义神经网络

Michigan Repetitive Knowledge Library,简称 MRKL,是密歇根大学研发的一种人工智能技术。它主要用于训练和部署在线的智能Agent,完成各种自然语言生成任务。
MRKL 的主要思想是:
1. 使用预训练的大型语言模型,如GPT-3,BERT等。这些模型已经在海量的数据上预训练,具有很强的语言理解和生成能力。
2. 通过“语言模型链技术”(Language Model Chaining)将多个语言模型串联起来使用。这是MRKL的关键技术。它将单个模型的下一步预测作为下一个模型的输入,这样可以在整个模型链上维持一致的上下文,产生连贯和长期依赖性强的生成文本。
3. 可以对预训练语言模型进行定制和微调,以满足不同的生成任务需求。这使得MRKL可以应用于各种NLG任务,如自动摘要,对话,问答生成等。
4. MRKL 系统本身也是一种AI Agent。它可以接收输入,根据输入和上下文生成相应的自然语言输出。研究人员和开发者可以训练定制的MRKL Agent来完成各种NLG任务。
所以,总结一下,MRKL 是密歇根大学研发的一种人工智能技术,它通过利用和组合大型预训练语言模型,并采用“语言模型链”技术,可以构建智能的在线Agent系统,完成各种自然语言生成任务。它是一种强大的NLG工具和方法。

MRKL Agent与普通的语言模型Agent(LLM Agent)有以下主要区别:
1. MRKL Agent 使用语言模型链技术,将多个语言模型串联使用。而普通的LLM Agent通常只使用单个语言模型。
2. MRKL Agent 可以在整个模型链上维持连贯和长期的上下文。由于使用多个模型,后续模型的输入包含前面模型的预测结果,所以可以捕捉到长期依赖关系。而普通LLM Agent每个预测都是单个模型的Fresh Start,难以建立长期上下文。
3. MRKL Agent可以利用不同的语言模型的优势,如使用BERT进行语义解析,使用GPT-3进行语言生成。而普通LLM Agent只能使用单一模型。
4. MRKL Agent易于定制和调整。由于是使用多个现成的语言模型,我们可以只调整某个模型或模型链的结构和参数来满足需求。而要定制一个单一的大型LLM需要大量数据和算力,难度较大。
5. MRKL Agent可以轻松应用于各种自然语言生成任务,效果较好。由于其长期上下文和多模型的特点,MRKL Agent在自动摘要,对话,故事生成等任务上会有比较好的表现。而普通LLM Agent效果可能不及。
所以,总体来说,MRKL Agent通过语言模型链技术和多模型组合的优势,在自然语言生成能力,长期依赖建模,定制灵活性等方面都优于普通的单一语言模型Agent。它是一种非常有潜力的新型NLG工具。



三个组成部分
Tools 
LLMChain 
Agent class itself 

重要的是prompt，构建输入输出公式format，用agent_scratchpad记录上一步的行动和结果actions/observation .
也可以根据这些原理，做你自己的prompt . 推荐使用ZeroShotAgent，加上以下参数
tools:可用的工具列表
prefix: 前缀
suffix: 后缀
input_varibles: 输入项

In [7]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain import OpenAI, SerpAPIWrapper, LLMChain 

In [10]:
class fakefuc():
    def run():
        print("fake tool running")


search = fakefuc()
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    )
]

In [13]:
prefix = """尽你所能回答问题, 用兄弟哥们儿的语气. 你可以使用的工具如下:"""
suffix = """开始! 记住，用兄弟哥们儿的语气讲话，多用"哈哈" "

Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools, 
    prefix=prefix, 
    suffix=suffix, 
    input_variables=["input", "agent_scratchpad"]
)

In [ ]:
print(prompt.template)

In [ ]:
llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=prompt)

In [ ]:
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)

In [ ]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.run("How many people live in canada as of 2023?")

# Multiple inputs
有多个输入的情况
增加输入项，就等于在prompt模板中增加内容。把增加的项嵌入到模板适当的位置即可。

In [ ]:
# 增加一个 language 输入项
prefix = """Answer the following questions as best you can. You have access to the following tools:"""
suffix = """When answering, you MUST speak in the following language: {language}.

Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools, 
    prefix=prefix, 
    suffix=suffix, 
    input_variables=["input", "language", "agent_scratchpad"] # 增加一个 language 输入项
)

llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=prompt)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools)

agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)
agent_executor.run(input="How many people live in canada as of 2023?", language="italian")# 增加一个 language 输入项

# MultiAction Agent
有多个行动选择的情况

fakeAgent这个例子，没有看懂奥妙。好像只是说用Agent选择工具，如何选择却没有说。
Tool Retrieval 这个例子，用向量相似的方式，来选择工具。但是，这只能调用一次工具。只能应付简单的任务。

In [20]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain import OpenAI, SerpAPIWrapper, LLMChain
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish
import re

def random_word(query: str) -> str:
    print("\nNow I'm doing this!")
    return "foo"

def fake_search(url:str) ->str:
    print ("\nget "+url ) 
    return "\nget "+url 

def fake_func(inp: str) -> str:
    return "foo"

fake_tools = [
    Tool(
        name=f"foo-{i}", 
        func=fake_func, 
        description=f"a silly function that you can use to get more information about the number {i}"
    ) 
    for i in range(99)
]
# search =  SerpAPIWrapper()
ALL_TOOLS  = [
    Tool(
        name = "Search",
        func=fake_search,#search.run,
        description="useful for when you need to answer questions about current events"
    ),
    Tool(
        name = "RandomWord",
        func=random_word,
        description="call this to get a random word."
    
    )
] + fake_tools

In [27]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
import os
#os.environ["OPENAI_API_KEY"] = "sk-写在.env中了" 

docs = [Document(page_content=t.description, metadata={"index": i}) for i, t in enumerate(ALL_TOOLS)]

In [ ]:
vector_store = FAISS.from_documents(docs, OpenAIEmbeddings())  

retriever = vector_store.as_retriever()

def get_tools(query):
    docs = retriever.get_relevant_documents(query)
    return [ALL_TOOLS[d.metadata["index"]] for d in docs]

In [ ]:
get_tools("whats the weather?")

In [ ]:
get_tools("whats the number 13?")

In [ ]:
# Set up the base template
template = """Answer the following questions as best you can, but speaking as a pirate might speak. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to speak as a pirate when giving your final answer. Use lots of "Arg"s

Question: {input}
{agent_scratchpad}"""

In [ ]:
from typing import Callable
# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    ############## NEW ######################
    # The list of tools available
    tools_getter: Callable
    
    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        ############## NEW ######################
        tools = self.tools_getter(kwargs["input"])
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in tools])
        return self.template.format(**kwargs)

In [ ]:
prompt = CustomPromptTemplate(
    template=template,
    tools_getter=get_tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"]
)

In [ ]:
class CustomOutputParser(AgentOutputParser):
    
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

In [ ]:
llm = OpenAI(temperature=0)
# LLM chain consisting of the LLM and a prompt
llm_chain = LLMChain(llm=llm, prompt=prompt)

tools = get_tools("whats the weather?")
tool_names = [tool.name for tool in tools]

agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    stop=["\nObservation:"], 
    allowed_tools=tool_names
)

In [ ]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.run("What's the weather in SF?")